In [1]:
using DataFrames, CSV, Statistics, HypothesisTests, CategoricalArrays

In [2]:
df = CSV.read("../data/trial-numbers.csv", DataFrame; missingstring="NA", types=Dict(7 => Int), silencewarnings=true)
transform!(df, Not("Trial #") .=> categorical; renamecols=false)
rename!(df, "Subject code" => :subject, "Session" => :session, "Trial type" => :type, "Cal type" => :calib, "Task" => :task, "Cue" => :cue, "Trial #" => :trial)
sort!(df, :trial);

In [3]:
count(x -> x .== "DATA", df.type)

288

In [4]:
gd = groupby(subset(df, :type => x -> x .== "DATA"), [:subject, :session, :task])
cf = combine(gd, :cue => (x -> first(x) == "CUED") => :cued_first);

In [5]:
BinomialTest(cf.cued_first)

Binomial test
-------------
Population details:
    parameter of interest:   Probability of success
    value under h_0:         0.5
    point estimate:          0.551724
    95% confidence interval: (0.467, 0.6343)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.2449

Details:
    number of observations: 145
    number of successes:    80


In [6]:
BinomialTest(cf[cf.task .== "RRT", :cued_first])

Binomial test
-------------
Population details:
    parameter of interest:   Probability of success
    value under h_0:         0.5
    point estimate:          0.534247
    95% confidence interval: (0.4137, 0.652)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.6400

Details:
    number of observations: 73
    number of successes:    39


In [7]:
BinomialTest(cf[cf.task .== "SIP", :cued_first])

Binomial test
-------------
Population details:
    parameter of interest:   Probability of success
    value under h_0:         0.5
    point estimate:          0.569444
    95% confidence interval: (0.4473, 0.6857)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.2888

Details:
    number of observations: 72
    number of successes:    41


In [8]:
gd = groupby(subset(df, :type => x -> x .== "DATA"), [:subject, :session])
tf = combine(gd, [:task, :cue] => ((t, c) -> length(t) < 2 ? (;RRT_first=missing, split=missing) : (;RRT_first=first(t)=="RRT", split=!all(==(t[1]), t[1:2]))) => AsTable);

In [9]:
any(skipmissing(tf.split))

false

In [10]:
BinomialTest(collect(skipmissing(tf.RRT_first)))

Binomial test
-------------
Population details:
    parameter of interest:   Probability of success
    value under h_0:         0.5
    point estimate:          0.458333
    95% confidence interval: (0.3402, 0.58)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.5560

Details:
    number of observations: 72
    number of successes:    33
